# Project 2

## Imports

In [2]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
import requests
import time, os

In [99]:
# Import Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [10]:
# Designate a URL to pull data from
#url = 'https://store.steampowered.com/search/?category1=998'
url = 'https://store.steampowered.com/app/289070/Sid_Meiers_Civilization_VI/'
response  = requests.get(url)

In [101]:
# Check if the site is scrapable; it is (b/c output is 200)!
# response.status_code

200

In [106]:
# Use the chromedriver to access the website
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [11]:
soup = BeautifulSoup(response.content, 'lxml')

In [109]:
game_link = driver.find_element_by_xpath("//*[contains(text(), 'Let’s go')]")
game_link.click()
time.sleep(1)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=83.0.4103.116)


In [110]:
# Make a list of sites to go through
list_of_links = []

# Retrieve a list of links
for link in soup.find('div', id = 'search_resultsRows').find_all('a', href = True):
    list_of_links.append(link.get('href'))
    
list_of_links

['https://store.steampowered.com/app/1085660/Destiny_2/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/976730/Halo_The_Master_Chief_Collection/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/782330/DOOM_Eternal/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/381210/Dead_by_Daylight/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/289070/Sid_Meiers_Civilization_VI/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/582010/Monster_Hunter_World/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/1174180/Red_Dead_Redemption_2/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/359550/Tom_Clancys_Rainbow_Six_Siege/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/526870/Satisfactory/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/app/397540/Borderlands_3/?snr=1_7_7_230_150_1',
 'https://store.steampowered.com/

In [143]:
# Data from a SPECIFIC site

import pandas as pd

df = pd.DataFrame()

with open("steamGameList.txt", "r") as file:
    for line in file:
        url = line.split("\n")[0]
        
        
        response  = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')

        # Title
        title = soup.find('div', class_ = 'apphub_AppName').text

        # Release Date
        release_date = soup.find('div', class_ = 'date').text

        # Price: will collect discounted prices and other text; must be cleaned in dataframe
        price = soup.find('div', class_ = 'game_purchase_action_bg').text.strip()

        # Make empty genre to append multiple genres
        genre = []
        developer = []
        publisher = []

        # Checks for specific keywords in line; then makes a list (in case multiple genres/devs/pubs)
        for detail in soup.find('div', class_ = 'details_block').find_all('a'):
            if 'genre' in detail.get('href'):
                genre.append(detail.getText())
            elif 'developer' in detail.get('href'):
                developer.append(detail.getText())
            elif 'publisher' in detail.get('href'):
                publisher.append(detail.getText())
                
        # Create a PRIMARY genre that's listed at the TOP LEFT of the Game Page for reference (may be needed later)
        primary_genre = soup.find('div', class_ = 'breadcrumbs').find_all('a')[1].text

        # Collects a string for the ratings (recent and all); to be cleaned in dataframe
        for rating in soup.find('div', class_ = 'user_reviews').find_all('span', class_ = 'nonresponsive_hidden responsive_reviewdesc'):
            if 'reviews in the last' in rating.text:
                recent_reviews = rating.getText().strip()
            else:
                all_reviews = rating.getText().strip()

        # Collect Metacritic score; some games do not have a score and will print 'No Score'
        try:
            metacritic_score = soup.find('div', class_ = 'score high').text.strip()
        except:
            metacritic_score = "No Score"

        game_dictionary = {'Title': title, 'Release_Date': release_date, 'Price': price, 'Genre': genre, 'Developer': developer, 'Publisher': publisher, 'Primary_Genre': primary_genre, 'Recent_Reviews': recent_reviews, 'All_Reviews': all_reviews, 'Metacritic': metacritic_score}

        df = df.append(game_dictionary, ignore_index = True)

KeyboardInterrupt: 

In [144]:
print(primary_genre)

Action Games


In [135]:
game_dictionary

{'Title': 'Red Dead Redemption 2',
 'Release_Date': 'Dec 5, 2019',
 'Price': '-20%$59.99$47.99 \n\nAdd to Cart',
 'Genre': ['Action', 'Adventure'],
 'Developer': ['Rockstar Games'],
 'Publisher': ['Rockstar Games'],
 'Recent_Reviews': '- 87% of the 6,286 user reviews in the last 30 days are positive.',
 'All_Reviews': '- 76% of the 83,356 user reviews for this game are positive.',
 'Metacritic': '93'}

In [145]:
# KEEP THIS LINE TO REFERENCE HOW TO OPEN THE FILE
with open("steamGameList.txt", "r") as file:
    for line in file:
        print (line.split("\n")[0])

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [36]:
# Imports
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
# Initiate a dataframe
df = pd.DataFrame()

with open("steamGameList.txt", "r") as file:
    for line in file:
        try:
            url = line.split("\n")[0]

            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'lxml')

            # Title
            title = soup.find('div', class_='apphub_AppName').text

            # Release Date
            release_date = soup.find('div', class_='date').text

            # Price: will collect discounted prices and other text; must be cleaned in dataframe
            price = soup.find(
                'div', class_='game_purchase_action_bg').text.strip()
            
            # Storage: this will pull storage from Windows (min, then recommend), Mac, and Linux (if compatible)
            storage = []

            for requirement in soup.find('div', class_ = 'sysreq_contents').find_all('li'):
                if 'Storage' in requirement.getText():
                    storage.append(requirement.getText())

            # Make empty genre to append multiple genres
            genre = []
            developer = []
            publisher = []

            # Checks for specific keywords in line; then makes a list (in case multiple genres/devs/pubs)
            for detail in soup.find('div', class_='details_block').find_all('a'):
                if 'genre' in detail.get('href'):
                    genre.append(detail.getText())
                elif 'developer' in detail.get('href'):
                    developer.append(detail.getText())
                elif 'publisher' in detail.get('href'):
                    publisher.append(detail.getText())

            # Create a PRIMARY genre that's listed at the TOP LEFT of the Game Page for reference (may be needed later)
            primary_genre = soup.find(
                'div', class_='breadcrumbs').find_all('a')[1].text

            # Collects a string for the ratings (recent and all); to be cleaned in dataframe
            for rating in soup.find('div',
                                    class_='user_reviews').find_all('span', class_='nonresponsive_hidden responsive_reviewdesc'):
                if 'reviews in the last' in rating.text:
                    recent_reviews = rating.getText().strip()
                else:
                    all_reviews = rating.getText().strip()

            # Collect Metacritic score; some games do not have a score and will print 'No Score'
            try:
                metacritic_score = soup.find(
                    'div', class_='score high').text.strip()
            except:
                metacritic_score = "No Score"

            game_dictionary = {'Title': title, 'Release_Date': release_date, 'Price': price,
                               'Genre': genre, 'Developer': developer, 'Publisher': publisher,
                               'Primary_Genre': primary_genre, 'Recent_Reviews': recent_reviews,
                               'All_Reviews': all_reviews, 'Metacritic': metacritic_score, 'Storage': storage[0]}

            df = df.append(game_dictionary, ignore_index=True)
            
            if len(df) % 5 == 0:
                df.to_csv('steam_dataframe_test.csv')

            if len(df) > 12500:
                break

        except:
            continue

df.to_csv('steam_dataframe_test.csv')

In [34]:
# Storage (based on system requirement)
storage = []

for requirement in soup.find('div', class_ = 'sysreq_contents').find_all('li'):
    if 'Storage' in requirement.getText():
        storage.append(requirement.getText())
        print(requirement.getText())
             
    #if 'storage' in requirement.get('href'):

Storage: 12 GB available space
Storage: 12 GB available space
Storage: 15 GB available space
Storage: 15 GB available space


In [33]:
storage

'Storage: 12 GB available space'